In [250]:
import pandas as pd
import json
from geopy.distance import geodesic
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point
import osmnx as ox
import contextily as ctx
from matplotlib.lines import Line2D
import unicodedata

# Pistes_cyclables

In [115]:
#Importer
df_exclusive = pd.read_csv('pistes_cyclables.csv')

In [139]:
# Fonction pour convertir la colonne 'geo_shape' en objets géométriques
def parse_geo_shape(geo_shape):
    try:
        shape_data = json.loads(geo_shape)
        if shape_data['type'] == 'LineString':
            return LineString(shape_data['coordinates'])
    except (json.JSONDecodeError, KeyError, TypeError):
        return None

# Conversion des données 'geo_shape' en géométrie
df_exclusive['geometry'] = df_exclusive['geo_shape'].apply(parse_geo_shape)

# Création d'un GeoDataFrame
gdf_exclusive = gpd.GeoDataFrame(df_exclusive, geometry='geometry')

# Suppression des lignes sans géométrie valide
gdf_exclusive = gdf_exclusive[gdf_exclusive['geometry'].notnull()]

# Définir le CRS initial 
gdf_exclusive = gdf_exclusive.set_crs(epsg=4326)
print(gdf_exclusive.crs)
gdf_exclusive = gdf_exclusive.to_crs(epsg=3857)
print(gdf_exclusive.crs)

EPSG:4326
EPSG:3857


In [141]:
# Apperçu
gdf_exclusive.head()

,ogc_fid,geo_point_2d,anne_maj,lametro,type,name,geo_shape,geometry
0,1489,"45.17402,5.74223",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.74223,4...","LINESTRING (639222.12 5648959.104, 639120.819 ..."
1,1490,"45.17272,5.73701",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.73701,4...","LINESTRING (638641.032 5648753.824, 638539.731..."
2,1491,"45.17273,5.73488",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.73488,4...","LINESTRING (638403.921 5648755.403, 638398.355..."
3,1492,"45.17315,5.73476",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.73476,4...","LINESTRING (638390.563 5648821.724, 638363.846..."
4,1625,"45.20183,5.75957",2016.0,1,chronovelo,NaN,"{""type"":""LineString"",""coordinates"":[[5.75957,4...","LINESTRING (641152.4 5653351.65, 641253.7 5653..."


In [219]:
# Exporter 
gdf_exclusive.to_csv("Pistes.csv", index=False)

# Données INSEE

In [28]:
df_insee = pd.read_csv('DS_RP_SERIE_HISTORIQUE_data.csv', sep=';', encoding='latin1', low_memory=False)

In [30]:
# Liste des codes à garder
codes_a_garder = [
    38423, 38169, 38151, 38421, 38317,
    38158, 38309, 38485, 38229, 38516,
    38382, 38185
]

# Conversion de la colonne GEO en numérique (si nécessaire)
df_insee['GEO'] = pd.to_numeric(df_insee['GEO'], errors='coerce')

# Extraire les lignes contenant les codes à garder
df_sample = df_insee[df_insee['GEO'].isin(codes_a_garder)]

In [32]:
# Ne garder que les communes
df_sample = df_sample[df_sample['GEO_OBJECT'] == 'COM']

In [34]:
# Ne garder que les résidences principales
df_sample = df_sample[df_sample['OCS'] == 'DW_MAIN']

In [38]:
# Ne garder que la Population des ménages
df_sample = df_sample[df_sample['RP_MEASURE'] == 'DWELLINGS_POPSIZE']

In [40]:
# Ne garder que la dernière année
df_sample = df_sample[df_sample['TIME_PERIOD'] == 2021]

In [44]:
# Dictionnaire des codes et des noms de villes
codes_villes = {
    38185: "Grenoble",
    38423: "Saint-Martin-le-Vinoux",
    38169: "Fontaine",
    38151: "Echirolles",
    38421: "Saint-Martin-d’Hères",
    38317: "Le Pont-de-Claix",
    38158: "Eybens",
    38309: "Poisat",
    38485: "Seyssinet-Pariset",
    38229: "Meylan",
    38516: "La Tronche",
    38382: "Saint-Egrève"
}

# Ajouter une colonne avec les noms des villes dans df_sample
df_sample['Ville'] = df_sample['GEO'].map(codes_villes)

# Afficher les correspondances entre les noms des villes et les codes
print(df_sample)

             GEO GEO_OBJECT FREQ         RP_MEASURE      OCS  TIME_PERIOD  \
1947577  38309.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1948529  38158.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1948545  38185.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1949149  38229.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1950235  38151.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1951111  38169.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1951733  38516.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1951768  38317.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1952090  38423.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1952250  38421.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1954402  38485.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   
1955602  38382.0        COM    A  DWELLINGS_POPSIZE  DW_MAIN         2021   

In [125]:
# Liste des communes à afficher
communes_cibles = [
    "Grenoble",
    "Saint-Martin-le-Vinoux",
    "Fontaine",
    "Echirolles",
    "Saint-Martin-d’Hères",
    "Le Pont-de-Claix",
    "Eybens",
    "Poisat",
    "Seyssinet-Pariset",
    "Meylan",
    "La Tronche",
    "Saint-Egrève"
]

# Création d'un GeoDataFrame vide pour stocker les communes
communes_selectionnees = gpd.GeoDataFrame()

# Télécharger les limites administratives pour chaque commune et les ajouter au GeoDataFrame
for commune in communes_cibles:
    try:
        commune_gdf = ox.geocode_to_gdf(commune + ", France")  # Récupérer les limites pour chaque commune
        commune_gdf = commune_gdf.to_crs(epsg=3857)  # Convertir au CRS EPSG:3857
        communes_selectionnees = gpd.GeoDataFrame(
            pd.concat([communes_selectionnees, commune_gdf], ignore_index=True),
            crs=commune_gdf.crs
        )
    except Exception as e:
        print(f"Erreur lors du traitement de {commune}: {e}")


In [155]:
# Effectuer une jointure spatiale pour associer les pistes aux communes qu'elles traversent
pistes_avec_communes = gpd.sjoin(gdf_exclusive, communes_selectionnees, how="inner", predicate="intersects")

# Créer un GeoDataFrame pour stocker les segments découpés
segments = []

# Parcourir chaque ligne de la jointure spatiale
for idx, row in pistes_avec_communes.iterrows():
    # Géométrie de la piste cyclable
    piste_geom = row['geometry']
    # Géométrie de la commune (polygone)
    commune_geom = communes_selectionnees.loc[row['index_right'], 'geometry']
    # Intersection entre la piste cyclable et la commune
    intersection = piste_geom.intersection(commune_geom)
    # Si l'intersection est une géométrie valide (ex. : LineString)
    if isinstance(intersection, LineString):
        # Créer une nouvelle entrée pour le segment découpé
        new_row = row.copy()
        new_row['geometry'] = intersection
        segments.append(new_row)

# Créer un nouveau GeoDataFrame contenant les segments découpés
segments_gdf = gpd.GeoDataFrame(segments, columns=pistes_avec_communes.columns, crs=gdf_exclusive.crs)

# Résultat : chaque segment de piste cyclable est associé à une seule commune
print(segments_gdf.head())
print(segments_gdf.crs)

   ogc_fid      geo_point_2d  anne_maj  lametro   type_left name_left  \
0     1489  45.17402,5.74223    2016.0        1  chronovelo       NaN   
1     1490  45.17272,5.73701    2016.0        1  chronovelo       NaN   
2     1491  45.17273,5.73488    2016.0        1  chronovelo       NaN   
3     1492  45.17315,5.73476    2016.0        1  chronovelo       NaN   
4     1625  45.20183,5.75957    2016.0        1  chronovelo       NaN   

                                           geo_shape  \
0  {"type":"LineString","coordinates":[[5.74223,4...   
1  {"type":"LineString","coordinates":[[5.73701,4...   
2  {"type":"LineString","coordinates":[[5.73488,4...   
3  {"type":"LineString","coordinates":[[5.73476,4...   
4  {"type":"LineString","coordinates":[[5.75957,4...   

                                            geometry  index_right  bbox_west  \
0  LINESTRING (639222.12 5648959.104, 639120.819 ...            0   5.677606   
1  LINESTRING (638641.032 5648753.824, 638539.731...            

In [159]:
# Calculer la distance de chaque segment de piste cyclable en kilomètres
segments_gdf['Distance'] = segments_gdf['geometry'].length / 1000  # Convertir la longueur en km

# Vérifier les résultats
print(segments_gdf[['geometry', 'Distance']].head())

                                            geometry  Distance
0  LINESTRING (639222.12 5648959.104, 639120.819 ...  0.239981
1  LINESTRING (638641.032 5648753.824, 638539.731...  0.237120
2  LINESTRING (638403.921 5648755.403, 638398.355...  0.074430
3  LINESTRING (638390.563 5648821.724, 638363.846...  0.109306
4  LINESTRING (641152.4 5653351.65, 641253.7 5653...  0.276608


In [221]:
# Exporter 
segments_gdf.to_csv("Pistes_segmentees.csv", index=False)

In [223]:
# Grouper par 'display_name' (le nom de la commune) et calculer la somme des distances
commune_pistes = segments_gdf.groupby('name_right').agg(
    Km_de_pistes=('Distance', 'sum')  # Somme des distances pour chaque commune
).reset_index()

# Renommer la colonne pour plus de clarté
commune_pistes.rename(columns={'name_right': 'Commune'}, inplace=True)

# Vérification des résultats
print(commune_pistes.head())

            Commune  Km_de_pistes
0            Eybens     21.566537
1          Fontaine     34.667450
2          Grenoble    147.576766
3        La Tronche     17.228974
4  Le Pont-de-Claix     12.628376


In [225]:
# Modifier la fonction pour extraire et nettoyer les noms simples
def extraire_nom_simple(display_name):
    # Extraire le nom simple avant la virgule
    nom_simple = display_name.split(",")[0]
    # Supprimer les accents (é, è, É, etc.) et normaliser les apostrophes
    nom_simple_sans_accents = ''.join(
        c for c in unicodedata.normalize('NFD', nom_simple) if unicodedata.category(c) != 'Mn'
    )
    # Remplacer les apostrophes typographiques par une simple apostrophe
    nom_simple_sans_accents = nom_simple_sans_accents.replace("’", "'")
    return nom_simple_sans_accents

# Appliquer cette fonction à la colonne 'Commune' (anciennement 'display_name')
commune_pistes['Commune_simplifiee'] = commune_pistes['Commune'].apply(extraire_nom_simple)

# Nettoyer également les noms des villes dans df_sample
df_sample['Ville_simplifiee'] = df_sample['Ville'].apply(extraire_nom_simple)

# Créer un dictionnaire pour mapper les villes simplifiées à leur population
habitants_dict = df_sample.set_index('Ville_simplifiee')['OBS_VALUE'].to_dict()

# Ajouter la colonne 'Nbr_habitants' au DataFrame 'commune_pistes' en mappant les noms simplifiés
commune_pistes['Nbr_habitants'] = commune_pistes['Commune_simplifiee'].map(habitants_dict)

# Supprimer la colonne temporaire si elle n'est plus nécessaire
commune_pistes.drop(columns=['Commune_simplifiee'], inplace=True)

# Vérifier les résultats
print(commune_pistes.head())

            Commune  Km_de_pistes  Nbr_habitants
0            Eybens     21.566537     9849.99902
1          Fontaine     34.667450    22802.57314
2          Grenoble    147.576766   154445.99597
3        La Tronche     17.228974     5737.53344
4  Le Pont-de-Claix     12.628376    10618.99429


In [227]:
# Ajouter une colonne 'Km/habitants' dans le DataFrame 'commune_pistes'
commune_pistes['Km/habitants'] = commune_pistes['Km_de_pistes'] / commune_pistes['Nbr_habitants']

In [229]:
# Création du dictionnaire avec les villes et leurs superficies
villes_valeurs = {
    "Eybens": 4.5,
    "Fontaine": 6.7,
    "Grenoble": 18.13,
    "La Tronche": 6.4,
    "Le Pont-de-Claix": 5.6,
    "Meylan": 12.3,
    "Poisat": 2.6,
    "Saint-Martin-d'Hères": 9.3,
    "Saint-Martin-le-Vinoux": 10.1,
    "Saint-Égrève": 10.9,
    "Seyssinet-Pariset": 10.65,
    "Échirolles": 7.9
}

In [231]:
# Ajouter une colonne 'Commune_seule' qui contient uniquement le premier élément de la colonne 'Commune'
commune_pistes['Commune_seule'] = commune_pistes['Commune'].apply(lambda x: x.split(",")[0])

In [233]:
# Ajouter une colonne 'Superficie' 
commune_pistes['Superficie'] = commune_pistes['Commune_seule'].map(villes_valeurs)

In [235]:
# Ajouter une colonne 'Km/superficie' 
commune_pistes['Km/superficie'] = commune_pistes['Km_de_pistes'] / commune_pistes['Superficie']

In [237]:
# Ajouter une colonne 'Densité' 
commune_pistes['Densité'] = commune_pistes['Nbr_habitants'] / commune_pistes['Superficie']

In [239]:
# Ajouter une colonne 'Km/densité' 
commune_pistes['Km/densité'] = commune_pistes['Km_de_pistes'] / commune_pistes['Densité']

In [241]:
# Apperçu
commune_pistes.head()

,Commune,Km_de_pistes,Nbr_habitants,Km/habitants,Commune_seule,Superficie,Km/superficie,Densité,Km/densité
0,Eybens,21.566537,9849.99902,0.002189,Eybens,4.50,4.792564,2188.888671,0.009853
1,Fontaine,34.667450,22802.57314,0.001520,Fontaine,6.70,5.174246,3403.369125,0.010186
2,Grenoble,147.576766,154445.99597,0.000956,Grenoble,18.13,8.139921,8518.808382,0.017324
3,La Tronche,17.228974,5737.53344,0.003003,La Tronche,6.40,2.692027,896.489600,0.019218
4,Le Pont-de-Claix,12.628376,10618.99429,0.001189,Le Pont-de-Claix,5.60,2.255067,1896.248980,0.006660


In [243]:
# Exporter 
commune_pistes.to_csv("commune_pistes.csv", index=False)

# Comptages

In [189]:
#Importer 
df_comptages = pd.read_csv('comptages_velos_permanents.csv')

In [192]:
# Conserver uniquement les colonnes 'tmj_2021' et 'geo_point_2d'
df_comptages = df_comptages[['tmj_2022', 'geo_point_2d']]

In [200]:
# Calculer les rayons des cercles
facteur_echelle = 0.002
df_comptages['rayon'] = facteur_echelle * df_comptages['tmj_2022'] ** 0.5

In [202]:
# Inverser les coordonnées (lon, lat -> lat, lon) 
df_comptages['geometry'] = df_comptages['geo_point_2d'].apply(
    lambda x: Point(map(float, x.split(',')[::-1]))  # Inversion des coordonnées
)
df_comptages = gpd.GeoDataFrame(df_comptages, geometry='geometry', crs='EPSG:4326')
df_comptages = df_comptages.to_crs(epsg=3857)


In [204]:
# Apperçu
df_comptages.head()

,tmj_2022,geo_point_2d,rayon,geometry
0,184,"45.26632800001879, 5.883597000000001",0.027129,POINT (654959.022 5663547.281)
1,152,"45.21942400001893, 5.810544",0.024658,POINT (646826.799 5656131.704)
2,737,"45.15498351431255, 5.71241623986171",0.054295,POINT (635903.267 5645953.557)
3,59,"45.29204100001872, 5.633207999999999",0.015362,POINT (627085.846 5667615.136)
4,84,"45.32450782934831, 5.569027776990686",0.018330,POINT (619941.336 5672754.098)


In [206]:
# Exporter 
df_comptages.to_csv("comptages.csv", index=False)

# Arceaux

In [209]:
#Importer
df_arceaux = pd.read_csv('Arceaux.csv')

In [213]:
# Convertir df_arceaux en GeoDataFrame
df_arceaux[['latitude', 'longitude']] = df_arceaux['geo_point_2d'].str.split(',', expand=True)
df_arceaux['latitude'] = df_arceaux['latitude'].astype(float)
df_arceaux['longitude'] = df_arceaux['longitude'].astype(float)

df_arceaux['geometry'] = df_arceaux.apply(
    lambda row: Point(row['longitude'], row['latitude']), axis=1
)
df_arceaux = gpd.GeoDataFrame(df_arceaux, geometry='geometry', crs='EPSG:4326')
df_arceaux = df_arceaux.to_crs(epsg=3857)

# Calculer les rayons des cercles
facteur_echelle = 5  # Ajuster ce facteur pour modifier la taille des cercles
df_arceaux['rayon'] = facteur_echelle * df_arceaux['mob_arce_nb'] ** 0.5

In [215]:
# Apperçu
df_arceaux.head()

,mob_arce_id,geo_point_2d,mob_arce_nb,mob_arce_typ,mob_arce_datecre,latitude,longitude,geometry,rayon
0,46,"45.1865894781128,5.72616761182579",1,nouveau modèle,20010101000000,45.186589,5.726168,POINT (637434.063 5650944.168),5.000000
1,47,"45.1873270966455,5.72778228524969",2,nouveau modèle,20060101000000,45.187327,5.727782,POINT (637613.807 5651060.671),7.071068
2,48,"45.1875477425917,5.72837930646545",3,nouveau modèle,19990101000000,45.187548,5.728379,POINT (637680.267 5651095.522),8.660254
3,49,"45.1870128765722,5.72856041280374",5,nouveau modèle,20040101000000,45.187013,5.728560,POINT (637700.428 5651011.042),11.180340
4,50,"45.1876032980993,5.72890982744115",5,ancien modèle,19930101000000,45.187603,5.728910,POINT (637739.325 5651104.297),11.180340


In [246]:
# Exporter 
df_arceaux.to_csv("Arceaux2.csv", index=False)

In [248]:
df_arceaux.head()

,mob_arce_id,geo_point_2d,mob_arce_nb,mob_arce_typ,mob_arce_datecre,latitude,longitude,geometry,rayon
0,46,"45.1865894781128,5.72616761182579",1,nouveau modèle,20010101000000,45.186589,5.726168,POINT (637434.063 5650944.168),5.000000
1,47,"45.1873270966455,5.72778228524969",2,nouveau modèle,20060101000000,45.187327,5.727782,POINT (637613.807 5651060.671),7.071068
2,48,"45.1875477425917,5.72837930646545",3,nouveau modèle,19990101000000,45.187548,5.728379,POINT (637680.267 5651095.522),8.660254
3,49,"45.1870128765722,5.72856041280374",5,nouveau modèle,20040101000000,45.187013,5.728560,POINT (637700.428 5651011.042),11.180340
4,50,"45.1876032980993,5.72890982744115",5,ancien modèle,19930101000000,45.187603,5.728910,POINT (637739.325 5651104.297),11.180340
